In [1]:
import pandas as pd
import numpy as np
import re
import string
import nltk
from nltk.corpus import stopwords
nltk.download('stopwords')
from sentence_transformers import SentenceTransformer

from tensorflow.keras.models import Model
from tensorflow.keras.layers import Dense, LSTM, Input, Dropout, Activation, Conv1D, BatchNormalization
from tensorflow.keras.optimizers import Adadelta

[nltk_data] Downloading package stopwords to /home/pirate/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
/home/pirate/.local/share/virtualenvs/ai_prog_lec-IUo41nw4/lib/python3.10/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
2022-12-18 13:18:46.816624: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2022-12-18 13:18:48.604756: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer.so.7'; dlerror: libnvinfer.so.7: cannot open shared object file: No such file or directory


In [2]:
em_model = SentenceTransformer('/mnt/wine/ai_models/sentence_transformers/sentence-transformers_paraphrase-multilingual-mpnet-base-v2/')

/home/pirate/.local/share/virtualenvs/ai_prog_lec-IUo41nw4/lib/python3.10/site-packages/torch/cuda/__init__.py:88: UserWarning: CUDA initialization: CUDA unknown error - this may be due to an incorrectly set up environment, e.g. changing env variable CUDA_VISIBLE_DEVICES after program start. Setting the available devices to be zero. (Triggered internally at ../c10/cuda/CUDAFunctions.cpp:109.)
  return torch._C._cuda_getDeviceCount() > 0


In [3]:
stop_words = stopwords.words("english")
slang_stopwords = ["ive", "dont", "im", "amp", "thats", "doesnt", "havent", "hasnt", "theres", "shouldnt", "woman", "women", "man", "men", "mother"]
stop_words.extend(slang_stopwords)

In [4]:
data = pd.read_csv('labeled_data.csv')
data

,Unnamed: 0,count,hate_speech,offensive_language,neither,class,tweet
0,0,3,0,0,3,2,!!! RT @mayasolovely: As a woman you shouldn't...
1,1,3,0,3,0,1,!!!!! RT @mleew17: boy dats cold...tyga dwn ba...
2,2,3,0,3,0,1,!!!!!!! RT @UrKindOfBrand Dawg!!!! RT @80sbaby...
3,3,3,0,2,1,1,!!!!!!!!! RT @C_G_Anderson: @viva_based she lo...
4,4,6,0,6,0,1,!!!!!!!!!!!!! RT @ShenikaRoberts: The shit you...
...,...,...,...,...,...,...,...
24778,25291,3,0,2,1,1,you's a muthaf***in lie &#8220;@LifeAsKing: @2...
24779,25292,3,0,1,2,2,"you've gone and broke the wrong heart baby, an..."
24780,25294,3,0,3,0,1,young buck wanna eat!!.. dat nigguh like I ain...
24781,25295,6,0,6,0,1,youu got wild bitches tellin you lies


In [5]:
data = data[["tweet", "class"]]
data

,tweet,class
0,!!! RT @mayasolovely: As a woman you shouldn't...,2
1,!!!!! RT @mleew17: boy dats cold...tyga dwn ba...,1
2,!!!!!!! RT @UrKindOfBrand Dawg!!!! RT @80sbaby...,1
3,!!!!!!!!! RT @C_G_Anderson: @viva_based she lo...,1
4,!!!!!!!!!!!!! RT @ShenikaRoberts: The shit you...,1
...,...,...
24778,you's a muthaf***in lie &#8220;@LifeAsKing: @2...,1
24779,"you've gone and broke the wrong heart baby, an...",2
24780,young buck wanna eat!!.. dat nigguh like I ain...,1
24781,youu got wild bitches tellin you lies,1


In [6]:
def clean(text):
    text = text.lower()
    text = re.sub("https?://\w+\.\w+\.\w+", "", text) #removes all links
    text = re.sub('@[\w]+','',text) #removes usernames
    text = re.sub('#[\w]+','',text) #removes hashtags
    text = re.sub('rt','',text) #removes rt sign
    text = re.sub("[^a-z ]+", "", text) #removes all non-alphabetic characters
    return text



In [7]:
data['tweet'] = data['tweet'].apply(clean) 
data.head()

/tmp/ipykernel_3705623/1252549931.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data['tweet'] = data['tweet'].apply(clean)


,tweet,class
0,as a woman you shouldnt complain about clea...,2
1,boy dats coldtyga dwn bad for cuffin dat ho...,1
2,dawg you ever fuck a bitch and she sta to...,1
3,she look like a tranny,1
4,the shit you hear about me might be true or...,1


In [7]:
def remove_stopwords(text):
    text = text.split()
    text = [word for word in text if word not in stop_words]
    return " ".join(text)

In [ ]:
data['tweet'] = data['tweet'].apply(remove_stopwords)
data.head()

In [ ]:
data.to_csv('cleaned_data.csv', index=False)

In [8]:
def categergorical_to_binary(labels):
    for i, label in enumerate(labels):
        if label == 0:
            labels[i] = 1
        elif label == 2:
            labels[i] = 0
    return labels

In [9]:
# Split dataset into training and validation set
train_df = data.sample(frac=0.9)
val_df = data.drop(train_df.index)

# split text and labels
train_sentences = train_df["tweet"].to_numpy()
train_labels = train_df["class"].to_numpy()
val_sentences = val_df["tweet"].to_numpy()
val_labels = val_df["class"].to_numpy()

In [10]:
def get_embeddings(sentences):
    embs = []
    for sentence in sentences:
        embs.append(em_model.encode(sentence))
    return embs

In [11]:
train_embs = get_embeddings(train_sentences)
val_embs = get_embeddings(val_sentences)
train_embs = np.array(train_embs)
val_embs = np.array(val_embs)

In [12]:
train_embs.shape, val_embs.shape

((22305, 768), (2478, 768))

In [13]:
train_embs = np.expand_dims(train_embs, axis=1)
val_embs = np.expand_dims(val_embs, axis=1)
train_embs.shape, val_embs.shape

((22305, 1, 768), (2478, 1, 768))

In [14]:
train_labels = categergorical_to_binary(train_labels)
val_labels = categergorical_to_binary(val_labels)

In [15]:
from tensorflow.keras.layers import Bidirectional
def build_model():
    inputs = Input(name='the_input', shape=(1,768), dtype='float32')
    x = Dense(768, activation='relu')(inputs)
    x = Dropout(0.2)(x)
    x = Dense(384, activation='relu')(x)
    
    x = Conv1D(128, 5, padding='same')(x)
    x = BatchNormalization()(x)
    x = Activation('relu')(x)

    
    x = Dense(128, activation='relu')(x)
    x = Bidirectional(LSTM(128, return_sequences=True), merge_mode='sum')(x)
    x = Bidirectional(LSTM(64), merge_mode='concat')(x)
    x = BatchNormalization()(x)
    x = Dropout(0.2)(x)
    
    x = Dense(64, activation='relu')(x)
    x = Dropout(0.2)(x)
    x = Dense(32, activation='relu')(x)
    x = Dropout(0.2)(x)
    
    x = Dense(1, activation='sigmoid')(x)
    return Model(inputs=inputs, outputs=x)

In [17]:
model = build_model()
model.summary()

Model: "model_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 the_input (InputLayer)      [(None, 1, 768)]          0         
                                                                 
 dense_6 (Dense)             (None, 1, 768)            590592    
                                                                 
 dropout_4 (Dropout)         (None, 1, 768)            0         
                                                                 
 dense_7 (Dense)             (None, 1, 384)            295296    
                                                                 
 conv1d_1 (Conv1D)           (None, 1, 128)            245888    
                                                                 
 batch_normalization_2 (Batc  (None, 1, 128)           512       
 hNormalization)                                                 
                                                           

In [18]:
opt = Adadelta(learning_rate=1)
model.compile(loss='binary_crossentropy', optimizer=opt, metrics=['accuracy'])

In [19]:
EPOCHS = 10
history = model.fit(train_embs,
                    train_labels,
                    validation_data=(val_embs, val_labels),
                    verbose=1,
                    batch_size=16,
                    epochs=EPOCHS)

Epoch 1/10
1395/1395 [==============================] - 61s 34ms/step - loss: 0.2391 - accuracy: 0.9010 - val_loss: 0.1585 - val_accuracy: 0.9354
Epoch 2/10
1395/1395 [==============================] - 46s 33ms/step - loss: 0.1896 - accuracy: 0.9241 - val_loss: 0.1422 - val_accuracy: 0.9370
Epoch 3/10
1395/1395 [==============================] - 46s 33ms/step - loss: 0.1685 - accuracy: 0.9337 - val_loss: 0.1400 - val_accuracy: 0.9411
Epoch 4/10
1395/1395 [==============================] - 46s 33ms/step - loss: 0.1506 - accuracy: 0.9414 - val_loss: 0.1382 - val_accuracy: 0.9415
Epoch 5/10
1395/1395 [==============================] - 48s 34ms/step - loss: 0.1418 - accuracy: 0.9472 - val_loss: 0.1413 - val_accuracy: 0.9403
Epoch 6/10
1395/1395 [==============================] - 45s 32ms/step - loss: 0.1278 - accuracy: 0.9527 - val_loss: 0.1393 - val_accuracy: 0.9427
Epoch 7/10
1395/1395 [==============================] - 46s 33ms/step - loss: 0.1174 - accuracy: 0.9575 - val_loss: 0.1475 -

In [21]:
model.save_weights("nlpmodel.hdf5")

In [22]:
def predict(text):
    text = clean(text)
    #text = remove_stopwords(text)
    embeddings = em_model.encode([text])
    embeddings = np.expand_dims(embeddings, axis=1)
    pred = model.predict(embeddings)
    print(pred)
    if pred > 0.5:
        return "Hate speech/offensive language"
    else:
        return "normal"

In [32]:
predict("you are going to die tonight")

1/1 [==============================] - 0s 39ms/step
[[0.00020491]]


'normal'

In [ ]:
print(55)